# Feature Aggregator - Полный Pipeline для Global ML Model

**Цель**: Объединить все признаки и создать финальные датасеты для ML и Backtesting.

## Критические правила нормализации:

### ✅ ЧТО ВКЛЮЧАЕМ В ML ДАТАСЕТ:
- `log_return` - логарифмическая доходность
- `volume_zscore_*` - Z-score объема
- `dist_to_sma_*`, `dist_to_ema_*` - расстояние до MA в процентах
- `rsi_14` - RSI (0-100)
- `momentum_*` - нормализованный momentum
- `overnight_gap` - гэп открытия
- `day_of_week`, `day_of_month` - календарные признаки
- `ticker_id`, `sector_id` - метаданные для глобальной модели

### ❌ ЧТО НЕ ВКЛЮЧАЕМ В ML ДАТАСЕТ:
- `open`, `high`, `low`, `close` - абсолютные цены
- `volume` - абсолютный объем  
- `sma_*`, `ema_*` - абсолютные значения MA
- `vp_poc`, `vp_va_high`, `vp_va_low` - абсолютные уровни Volume Profile

## Выходные файлы:
- `data/processed_ml/{TICKER}_ml_features.parquet` - для ML
- `data/backtest/{TICKER}_price_data.parquet` - для бэктеста


In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к корневой директории ML
sys.path.insert(0, str(Path.cwd().parent))

# Импорт модулей признаков
from features import (
    build_all_features,
    process_all_tickers,
    get_ml_feature_columns,
    FORBIDDEN_ML_COLUMNS,
    VOLUME_FEATURE_COLUMNS,
    TREND_FEATURE_COLUMNS,
    CALENDAR_FEATURE_COLUMNS
)
from config import get_tickers_by_liquidity, get_ticker_metadata

import importlib
import features.feature_builder as fb
importlib.reload(fb)
from features import build_all_features

print("✅ Библиотеки и модули загружены")


✅ Библиотеки и модули загружены


## Конфигурация путей
-Скорее всего будет ошибка, поскольку данные берутся по другому пути

In [12]:
# === КОНФИГУРАЦИЯ ПУТЕЙ ===
ML_ROOT = Path.cwd().parent

# Входные данные (OHLCV с log_returns из этапа 01)
DATA_DIR = ML_ROOT / "data" / "processed"

# Выходные данные - ДВА отдельных каталога!
OUTPUT_ML_DIR = ML_ROOT / "data" / "processed_ml"      # Для ML модели (нормализованные)
OUTPUT_BACKTEST_DIR = ML_ROOT / "data" / "backtest"    # Для бэктеста (сырые цены)

# Создаем директории
OUTPUT_ML_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_BACKTEST_DIR.mkdir(parents=True, exist_ok=True)

print(f"📂 Источник данных: {DATA_DIR}")
print(f"📂 ML выход: {OUTPUT_ML_DIR}")
print(f"📂 Backtest выход: {OUTPUT_BACKTEST_DIR}")

# Проверяем доступные тикеры
available_files = list(DATA_DIR.glob("*_ohlcv_returns.parquet"))
available_tickers = [f.stem.replace('_ohlcv_returns', '') for f in available_files]
print(f"\n📋 Доступные тикеры ({len(available_tickers)}): {available_tickers}")


📂 Источник данных: e:\Python\VolatilityChecker\MOEXScanner\ML\data\processed
📂 ML выход: e:\Python\VolatilityChecker\MOEXScanner\ML\data\processed_ml
📂 Backtest выход: e:\Python\VolatilityChecker\MOEXScanner\ML\data\backtest

📋 Доступные тикеры (31): ['AFKS', 'AFLT', 'ALRS', 'BELU', 'BSPB', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'HYDR', 'IMOEX', 'IRAO', 'LENT', 'LKOH', 'MAGN', 'MGNT', 'MTSS', 'NLMK', 'NVTK', 'OZON', 'PIKK', 'PLZL', 'ROSN', 'RTKM', 'SBER', 'SNGS', 'TATN', 'TCSG', 'VKCO', 'VTBR', 'YNDX']


## Демонстрация на одном тикере


In [13]:
# Демонстрация на одном тикере
ticker = 'SBER'

# Загрузка исходных данных
df = pd.read_parquet(DATA_DIR / f"{ticker}_ohlcv_returns.parquet")
print(f"📥 Загружено: {len(df)} записей для {ticker}")
print(f"\nИсходные столбцы ({len(df.columns)}): {df.columns.tolist()}")

# Расчет ВСЕХ признаков
ml_features, backtest_data = build_all_features(df, ticker)

print(f"\n{'='*60}")
print(f"📊 РЕЗУЛЬТАТЫ ДЛЯ {ticker}")
print(f"{'='*60}")


📥 Загружено: 1301 записей для SBER

Исходные столбцы (8): ['date', 'open', 'high', 'low', 'close', 'volume', 'value', 'log_return']
  📊 Расчет признаков для SBER...
    • Объемные признаки...
    • Трендовые признаки...
    • Календарные признаки и Gap...
    ✅ Готово: 35 ML признаков, 6 Backtest столбцов

📊 РЕЗУЛЬТАТЫ ДЛЯ SBER


In [14]:
# === ML FEATURES (нормализованные) ===
print(f"✅ ML Features: {len(ml_features.columns)} столбцов")
print(f"\nСтолбцы ML: {ml_features.columns.tolist()}")
print(f"\nПример данных ML:")
display(ml_features.tail(10))

# Проверка: НЕ должно быть запрещенных столбцов!
forbidden_found = [col for col in ml_features.columns if col in FORBIDDEN_ML_COLUMNS]
if forbidden_found:
    print(f"\n❌ ОШИБКА: Найдены запрещенные столбцы: {forbidden_found}")
else:
    print(f"\n✅ Проверка пройдена: ML features не содержат абсолютных значений")


✅ ML Features: 35 столбцов

Столбцы ML: ['date', 'log_return', 'volume_zscore_20', 'volume_zscore_60', 'volume_ratio_20', 'volume_spike', 'vp_position', 'vp_width_pct', 'vp_above_va', 'dist_to_sma_20', 'dist_to_sma_50', 'dist_to_sma_200', 'dist_to_ema_20', 'dist_to_ema_50', 'sma_20_slope_norm', 'sma_50_slope_norm', 'momentum_10', 'momentum_20', 'rsi_14', 'price_position_ma', 'trend_signal', 'trend_strength', 'day_of_week', 'day_of_month', 'week_of_month', 'is_month_end', 'is_month_start', 'overnight_gap', 'overnight_gap_zscore', 'ticker_id', 'sector_id', 'sector_encoded', 'liquidity_rank', 'is_blue_chip', 'lot_size_log']

Пример данных ML:


,date,log_return,volume_zscore_20,volume_zscore_60,volume_ratio_20,volume_spike,vp_position,vp_width_pct,vp_above_va,dist_to_sma_20,...,is_month_end,is_month_start,overnight_gap,overnight_gap_zscore,ticker_id,sector_id,sector_encoded,liquidity_rank,is_blue_chip,lot_size_log
1291,2025-10-02 23:59:59,-0.004129,0.211341,0.458116,0.119234,0,-0.869754,0.073516,-1.0,-0.035968,...,0,0,0.000000,-0.349005,SBER,Finance,1,1,1,2.397895
1292,2025-10-03 23:59:59,-0.008132,-0.515931,-0.260651,-0.299846,0,-1.038450,0.068600,-1.0,-0.039849,...,0,0,0.001858,0.545986,SBER,Finance,1,1,1,2.397895
1293,2025-10-04 23:59:57,-0.008912,-1.193575,-1.015030,-0.752268,0,-1.252543,0.064686,-1.0,-0.044622,...,0,0,0.000318,-0.186137,SBER,Finance,1,1,1,2.397895
1294,2025-10-05 23:59:57,0.005654,-1.428724,-1.193376,-0.892507,0,-1.323263,0.057601,-1.0,-0.035780,...,0,0,0.000000,-0.323508,SBER,Finance,1,1,1,2.397895
1295,2025-10-06 23:59:55,0.029938,0.876131,1.031573,0.486507,0,-0.786819,0.056429,0.0,-0.004193,...,0,0,0.001418,0.358526,SBER,Finance,1,1,1,2.397895
1296,2025-10-07 23:59:54,0.011395,0.901245,1.104375,0.495490,0,0.257225,0.061368,0.0,0.008704,...,0,0,-0.001927,-1.209653,SBER,Finance,1,1,1,2.397895
1297,2025-10-08 23:59:51,-0.041653,2.656882,3.156851,1.706376,1,-0.498147,0.052346,0.0,-0.029133,...,0,0,0.000136,-0.215096,SBER,Finance,1,1,1,2.397895
1298,2025-10-09 23:59:55,0.026154,2.210263,3.064530,1.536564,1,0.019761,0.045289,0.0,-0.000803,...,0,0,0.003193,1.218827,SBER,Finance,1,1,1,2.397895
1299,2025-10-10 23:59:55,-0.014796,0.182137,0.774010,0.125304,0,-0.330539,0.044532,0.0,-0.012655,...,0,0,0.001832,0.554849,SBER,Finance,1,1,1,2.397895
1300,2025-10-11 15:00:27,-0.002388,-1.222483,-1.139398,-0.930690,0,-0.441381,0.038873,0.0,-0.013233,...,1,0,-0.002525,-1.469645,SBER,Finance,1,1,1,2.397895



✅ Проверка пройдена: ML features не содержат абсолютных значений


In [15]:
# === BACKTEST DATA (сырые цены) ===
print(f"✅ Backtest Data: {len(backtest_data.columns)} столбцов")
print(f"\nСтолбцы Backtest: {backtest_data.columns.tolist()}")
print(f"\nПример данных Backtest:")
display(backtest_data.tail(10))


✅ Backtest Data: 6 столбцов

Столбцы Backtest: ['date', 'open', 'high', 'low', 'close', 'volume']

Пример данных Backtest:


,date,open,high,low,close,volume
1291,2025-10-02 23:59:59,286.40,287.15,282.80,285.22,28023440
1292,2025-10-03 23:59:59,285.75,287.32,282.58,282.91,17098575
1293,2025-10-04 23:59:57,283.00,283.00,279.39,280.40,5632692
1294,2025-10-05 23:59:57,280.40,282.13,279.14,281.99,2450371
1295,2025-10-06 23:59:55,282.39,291.17,281.82,290.56,36480863
1296,2025-10-07 23:59:54,290.00,295.87,288.75,293.89,37893983
1297,2025-10-08 23:59:51,293.93,295.00,278.00,281.90,75222569
1298,2025-10-09 23:59:55,282.80,290.45,278.75,289.37,76179258
1299,2025-10-10 23:59:55,289.90,289.95,284.10,285.12,34168796
1300,2025-10-11 15:00:27,284.40,285.31,282.22,284.44,1976492


In [16]:
# Статистика по ML признакам
print("📈 Статистика ML признаков:")
display(ml_features.describe())


📈 Статистика ML признаков:


,date,log_return,volume_zscore_20,volume_zscore_60,volume_ratio_20,volume_spike,vp_position,vp_width_pct,vp_above_va,dist_to_sma_20,...,day_of_month,week_of_month,is_month_end,is_month_start,overnight_gap,overnight_gap_zscore,sector_encoded,liquidity_rank,is_blue_chip,lot_size_log
count,1301,1301.000000,1282.000000,1242.000000,1282.000000,1301.000000,1281.000000,1281.000000,1281.000000,1282.000000,...,1301.000000,1301.000000,1301.000000,1301.000000,1300.000000,1241.000000,1301.0,1301.0,1301.0,1.301000e+03
mean,2023-05-21 07:20:57.770177024,0.000247,-0.006485,-0.009050,0.009000,0.058417,0.113631,0.079088,0.116315,0.003220,...,15.935434,2.724827,0.046887,0.046887,0.000497,-0.017174,1.0,1.0,1.0,2.397895e+00
min,2020-10-13 23:59:59,-0.455918,-2.655441,-2.532335,-0.982280,0.000000,-2.739161,0.011234,-1.000000,-0.473190,...,1.000000,1.000000,0.000000,0.000000,-0.100719,-7.572504,1.0,1.0,1.0,2.397895e+00
25%,2022-01-24 23:59:59,-0.007616,-0.727783,-0.701396,-0.338029,0.000000,-0.367041,0.037562,0.000000,-0.016562,...,8.000000,2.000000,0.000000,0.000000,-0.000387,-0.310565,1.0,1.0,1.0,2.397895e+00
50%,2023-06-05 23:59:59,0.000539,-0.207430,-0.227375,-0.099342,0.000000,0.069039,0.054595,0.000000,0.006679,...,16.000000,3.000000,0.000000,0.000000,0.000607,0.023850,1.0,1.0,1.0,2.397895e+00
75%,2024-09-11 23:59:59,0.009146,0.470561,0.397295,0.201901,0.000000,0.599127,0.088736,1.000000,0.028326,...,24.000000,4.000000,0.000000,0.000000,0.002446,0.305527,1.0,1.0,1.0,2.397895e+00
max,2025-10-11 15:00:27,0.117686,4.065854,6.447254,3.573474,1.000000,4.048918,1.085826,1.000000,0.181725,...,31.000000,5.000000,1.000000,1.000000,0.063058,6.827133,1.0,1.0,1.0,2.397895e+00
std,NaN,0.023760,1.050420,1.082258,0.577842,0.234620,0.796054,0.099389,0.658875,0.059973,...,8.771395,1.268399,0.211478,0.211478,0.008314,0.988392,0.0,0.0,0.0,4.442600e-16


## Batch обработка всех тикеров


In [17]:
# Batch обработка ВСЕХ доступных тикеров
print("🚀 Запуск полного Feature Engineering Pipeline")
print(f"   Тикеры для обработки: {available_tickers}\n")

processed, errors = process_all_tickers(
    data_dir=DATA_DIR,
    output_ml_dir=OUTPUT_ML_DIR,
    output_backtest_dir=OUTPUT_BACKTEST_DIR,
    tickers=available_tickers
)

print(f"\n🏁 Pipeline завершен!")
print(f"   Успешно: {processed}/{len(available_tickers)}")
if errors:
    print(f"   Ошибки: {errors}")


🚀 Запуск полного Feature Engineering Pipeline
   Тикеры для обработки: ['AFKS', 'AFLT', 'ALRS', 'BELU', 'BSPB', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'HYDR', 'IMOEX', 'IRAO', 'LENT', 'LKOH', 'MAGN', 'MGNT', 'MTSS', 'NLMK', 'NVTK', 'OZON', 'PIKK', 'PLZL', 'ROSN', 'RTKM', 'SBER', 'SNGS', 'TATN', 'TCSG', 'VKCO', 'VTBR', 'YNDX']

📋 Обработка 31 тикеров...
   Тикеры: ['AFKS', 'AFLT', 'ALRS', 'BELU', 'BSPB', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'HYDR', 'IMOEX', 'IRAO', 'LENT', 'LKOH', 'MAGN', 'MGNT', 'MTSS', 'NLMK', 'NVTK', 'OZON', 'PIKK', 'PLZL', 'ROSN', 'RTKM', 'SBER', 'SNGS', 'TATN', 'TCSG', 'VKCO', 'VTBR', 'YNDX']

🔄 AFKS...
  📊 Расчет признаков для AFKS...
    • Объемные признаки...
    • Трендовые признаки...
    • Календарные признаки и Gap...
    ✅ Готово: 35 ML признаков, 6 Backtest столбцов
  💾 Сохранено: AFKS_ml_features.parquet, AFKS_price_data.parquet
🔄 AFLT...
  📊 Расчет признаков для AFLT...
    • Объемные признаки...
    • Трендовые признаки...
    • Календарные признаки и Gap...
    ✅ 

## Проверка результатов


In [18]:
# Проверка созданных файлов
print("📁 Файлы в processed_ml/:")
ml_files = list(OUTPUT_ML_DIR.glob("*.parquet"))
for f in ml_files:
    size_kb = f.stat().st_size / 1024
    print(f"   {f.name}: {size_kb:.1f} KB")

print(f"\n📁 Файлы в backtest/:")
backtest_files = list(OUTPUT_BACKTEST_DIR.glob("*.parquet"))
for f in backtest_files:
    size_kb = f.stat().st_size / 1024
    print(f"   {f.name}: {size_kb:.1f} KB")


📁 Файлы в processed_ml/:
   AFKS_ml_features.parquet: 254.7 KB
   AFLT_ml_features.parquet: 254.0 KB
   ALRS_ml_features.parquet: 254.4 KB
   BELU_ml_features.parquet: 250.7 KB
   BSPB_ml_features.parquet: 253.5 KB
   CHMF_ml_features.parquet: 254.2 KB
   FIVE_ml_features.parquet: 58.7 KB
   GAZP_ml_features.parquet: 254.9 KB
   GMKN_ml_features.parquet: 253.6 KB
   HYDR_ml_features.parquet: 253.8 KB
   IMOEX_ml_features.parquet: 210.2 KB
   IRAO_ml_features.parquet: 253.5 KB
   LENT_ml_features.parquet: 193.9 KB
   LKOH_ml_features.parquet: 254.7 KB
   MAGN_ml_features.parquet: 254.3 KB
   MGNT_ml_features.parquet: 254.7 KB
   MTSS_ml_features.parquet: 253.6 KB
   NLMK_ml_features.parquet: 254.4 KB
   NVTK_ml_features.parquet: 253.9 KB
   OZON_ml_features.parquet: 236.6 KB
   PIKK_ml_features.parquet: 254.5 KB
   PLZL_ml_features.parquet: 254.2 KB
   ROSN_ml_features.parquet: 254.7 KB
   RTKM_ml_features.parquet: 254.1 KB
   SBER_ml_features.parquet: 255.1 KB
   SNGS_ml_features.parqu

In [19]:
# Загрузка и проверка одного из созданных файлов
if ml_files:
    sample_file = ml_files[0]
    sample_df = pd.read_parquet(sample_file)
    print(f"📊 Пример: {sample_file.name}")
    print(f"   Строк: {len(sample_df)}")
    print(f"   Столбцов: {len(sample_df.columns)}")
    print(f"   Столбцы: {sample_df.columns.tolist()}")
    
    # Финальная проверка на запрещенные столбцы
    forbidden = [col for col in sample_df.columns if col in FORBIDDEN_ML_COLUMNS]
    if forbidden:
        print(f"\n❌ КРИТИЧЕСКАЯ ОШИБКА: В ML файле есть запрещенные столбцы: {forbidden}")
    else:
        print(f"\n✅ ML файл прошел валидацию - нет абсолютных значений цен/объема")


📊 Пример: AFKS_ml_features.parquet
   Строк: 1301
   Столбцов: 35
   Столбцы: ['date', 'log_return', 'volume_zscore_20', 'volume_zscore_60', 'volume_ratio_20', 'volume_spike', 'vp_position', 'vp_width_pct', 'vp_above_va', 'dist_to_sma_20', 'dist_to_sma_50', 'dist_to_sma_200', 'dist_to_ema_20', 'dist_to_ema_50', 'sma_20_slope_norm', 'sma_50_slope_norm', 'momentum_10', 'momentum_20', 'rsi_14', 'price_position_ma', 'trend_signal', 'trend_strength', 'day_of_week', 'day_of_month', 'week_of_month', 'is_month_end', 'is_month_start', 'overnight_gap', 'overnight_gap_zscore', 'ticker_id', 'sector_id', 'sector_encoded', 'liquidity_rank', 'is_blue_chip', 'lot_size_log']

✅ ML файл прошел валидацию - нет абсолютных значений цен/объема


## Сводка признаков

### Признаки для ML модели (всего ~30 признаков):

| Категория | Признаки | Описание |
|-----------|----------|----------|
| **Доходность** | `log_return` | Логарифмическая дневная доходность |
| **Объем** | `volume_zscore_20/60`, `volume_ratio_20`, `volume_spike` | Z-score и относительный объем |
| **Volume Profile** | `vp_position`, `vp_width_pct`, `vp_above_va` | Нормализованные метрики VP |
| **Тренд (MA)** | `dist_to_sma_20/50/200`, `dist_to_ema_20/50` | Расстояние до MA в % |
| **Наклон MA** | `sma_20_slope_norm`, `sma_50_slope_norm` | Нормализованный наклон |
| **Momentum** | `momentum_10`, `momentum_20` | Log return за период |
| **RSI** | `rsi_14` | Уже нормализован (0-100) |
| **Позиция цены** | `price_position_ma`, `trend_signal`, `trend_strength` | Категориальные/нормализованные |
| **Календарь** | `day_of_week`, `day_of_month`, `week_of_month` | Временные признаки |
| **Конец месяца** | `is_month_end`, `is_month_start` | Бинарные флаги |
| **Gap** | `overnight_gap`, `overnight_gap_zscore` | Гэп открытия |
| **Метаданные** | `ticker_id`, `sector_id`, `sector_encoded`, `liquidity_rank`, `is_blue_chip` | Для глобальной модели |
